In [1]:
import numpy as np
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
X_list = []
for filename in sorted(os.listdir('./Dataset_Resize/')):
    img = image.load_img(Path('./Dataset_Resize/', filename))
    X_list.append(image.img_to_array(img))
X = np.array(X_list)
print(X.shape)

df = pd.read_csv('./vehicles.csv', index_col=0)
df_dummy = pd.get_dummies(df['Vehicle_Type'])
Y = np.array(df_dummy)
print(Y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(9850, 299, 299, 3)
(9850, 6)
(7880, 299, 299, 3) (7880, 6)
(1970, 299, 299, 3) (1970, 6)


In [23]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adadelta, Adam
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping, LearningRateScheduler, ModelCheckpoint

we will only instantiate the convolutional part of the model, everything up to the fully-connected layers. <br>
We will then run this model on our training and validation data once, recording the output (the "bottleneck features" from Inception model: the last activation maps before the fully-connected layers) in two numpy arrays. <br>
Then we will train a small fully-connected model on top of the stored features<br>

In [7]:
batch_size = 256
datagen = ImageDataGenerator(rescale=1. / 255)
model_Inception = InceptionV3(include_top = False, weights = 'imagenet')   
print("load Inception V3")

load Inception V3


In [8]:
generator = datagen.flow(X_train, Y_train, 
                     batch_size=batch_size)

bottleneck_features_train = model_Inception.predict_generator(generator)
print("Calculate train features")
print(bottleneck_features_train.dtype)

Calculate train features
float32


In [9]:
np.savez('inception_features_train', features=bottleneck_features_train)
# np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)
print("Saved train features")

Saved train features


In [11]:
generator = datagen.flow(X_test, Y_test, 
                     batch_size=batch_size)

bottleneck_features_validation = model_Inception.predict_generator(generator)
print("Calculate test features")
print(bottleneck_features_validation.dtype)

Calculate test features
float32


In [12]:
np.savez('inception_features_validation', features=bottleneck_features_validation)
# np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)
print("Saved test features")

Saved test features


In [13]:
early_Stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    verbose=0, 
    mode='auto'
)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)
checkpointer = ModelCheckpoint(filepath="TL_Inception.hdf5", verbose=1, save_best_only = True)
csv = CSVLogger('TL_Inception.csv')

 load our saved data and train a small fully-connected model

In [17]:
features_train = np.load('inception_features_train.npz')['features']
print("done")
print(features_train.shape)

done
(7880, 8, 8, 2048)


In [18]:
features_validation = np.load('inception_features_validation.npz')['features']
print("done")
print(features_validation.shape)

done
(1970, 8, 8, 2048)


In [20]:
print(Y_train.shape, Y_test.shape)

(7880, 6) (1970, 6)


In [33]:
def model_add_top_layer(input_shape):  
    
    model_addtop = Sequential()
    model_addtop.add(GlobalAveragePooling2D(input_shape = input_shape))
    model_addtop.add(Dense(1024,activation='relu'))
    model_addtop.add(Dense(6, activation='softmax'))
    
    model_addtop.summary()
    
    opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
    
    model_addtop.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model_addtop

In [34]:
model_TL = model_add_top_layer(features_train.shape[1:])
history_InceptionV3TL = model_TL.fit(features_train, Y_train,
               batch_size=256,
               epochs=100,
               verbose=1,
               validation_data=(features_validation, Y_test),
               shuffle=True,
               callbacks=[early_Stopping,reduceLR,checkpointer,csv])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 6150      
Total params: 2,104,326
Trainable params: 2,104,326
Non-trainable params: 0
_________________________________________________________________
Train on 7880 samples, validate on 1970 samples
Epoch 1/100
7880/7880 [==============================] - 8s 1ms/step - loss: 1.5802 - acc: 0.5539 - val_loss: 1.2923 - val_acc: 0.6005

Epoch 00001: val_loss improved from 1.29791 to 1.29229, saving model to TL_Inception.hdf5
Epoch 2/100
7880/7880 [==============================] - 5s 693us/step - loss: 1.3253 - acc: 0.5829 - val_loss: 1.2999 - v